

# Low-level LSL API

.. include:: ./../../links.inc


LSL is a library designed for streaming time series data across different platforms and
programming languages. The [core library](lsl lib_) is primarily written in C++, and
bindings are accessible for Python, C#, Java, MATLAB, and Unity, among others. You can
find a comprehensive list [here](lsl language bindings_).

MNE-LSL provides a reimplementation of the [python binding](lsl python_), known as
``pylsl``, within the ``mne_lsl.lsl`` module. It introduces additional functionalities
to simplify the low-level interaction with LSL streams. Moreover, it enhances the
detection of liblsl on your system and can retrieve a compatible version online if
necessary. The differences between ``pylsl`` and ``mne_lsl.lsl`` are detailed
`here<resources/pylsl:Differences with pylsl>`.


In [2]:
import time
import uuid

from mne_lsl.lsl import (
    StreamInfo, StreamInlet, StreamOutlet, local_clock, resolve_streams
)

import mne
import os,pandas as pd
import numpy as np
import pathlib


import logging

# For interactive plots
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'qt')

# Define paths: 
current_path = pathlib.Path().absolute()  
recording_path = current_path / 'Recordings'


## Discover streams

At this point, the :class:`~mne_lsl.lsl.StreamOutlet` is available on the network. The
function :func:`~mne_lsl.lsl.resolve_streams` discovers all available streams on the
network. This operation is commonly named the stream resolution.

<div class="alert alert-info"><h4>Note</h4><p>The stream resolution can be restricted by providing the ``name``, ``stype``, and
    ``source_id`` arguments.</p></div>



In [15]:
streams = resolve_streams()

streams[:]

[< sInfo 'Tobii_open' >
   | Type: ET
   | Sampling: 600.0 Hz
   | Number of channels: 5
   | Data type: <class 'numpy.float32'>
   | Source: tet-tcp://169.254.10.243,
 < sInfo 'MyMarkerStream' >
   | Type: Markers
   | Sampling: Irregular
   | Number of channels: 1
   | Data type: string
   | Source: myuidw43536,
 < sInfo 'LiveAmpSN-101402-0868' >
   | Type: EEG
   | Sampling: 500.0 Hz
   | Number of channels: 67
   | Data type: <class 'numpy.float32'>
   | Source: RT_Sender_SimulationPC,
 < sInfo 'Tobii' >
   | Type: ET
   | Sampling: 600.0 Hz
   | Number of channels: 31
   | Data type: <class 'numpy.float32'>
   | Source: tet-tcp://169.254.10.243,
 < sInfo 'Prediction_stream' >
   | Type: Labels
   | Sampling: Irregular
   | Number of channels: 1
   | Data type: string
   | Source: 41af08]

The resolution retrieves only the stream basic properties. The channel properties,
stored in the stream description in an XML element tree, are absent from a
:class:`~mne_lsl.lsl.StreamInfo` returned by the resolution function.



In [16]:
assert streams[03].get_channel_names() is None

## Connect to a Stream

To connect to a stream, a :class:`~mne_lsl.lsl.StreamInlet` object must be created
using the resolved :class:`~mne_lsl.lsl.StreamInfo`. Once the stream is opened with
:meth:`~mne_lsl.lsl.StreamInlet.open_stream`, the connection is established and
both the properties and data become available.



In [22]:
inlet = StreamInlet(streams[2])
inlet.open_stream()
sinfo = inlet.get_sinfo()  # retrieve stream information with all properties

In [23]:
ch_names = sinfo.get_channel_names()

In [24]:
types = sinfo.get_channel_types()

In [25]:
sinfo.get_channel_units()

An :class:`mne.Info` can be obtained directly with
:meth:`~mne_lsl.lsl.StreamInfo.get_channel_info`. If the information contained in the
XML element tree can not be parsed, default values are used. For instance, the channel
names are replaced by the channel numbers similarly to :func:`mne.create_info`.



In [26]:
sinfo.get_channel_info()

[meas_info.create_info] WARNING: Something went wrong while reading the channel description. Defaulting to channel IDs and MNE-compatible stream type.


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,67 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,500.00 Hz
Highpass,0.00 Hz
Lowpass,0.50 Hz


## Creating a stream

To create a stream, you must first define its properties. This is achieved by creating
a :class:`~mne_lsl.lsl.StreamInfo` object, which specifies the stream's name, type,
source and properties. Convenience methods are available to set the channel
properties, including :meth:`~mne_lsl.lsl.StreamInfo.set_channel_info`, which uses a
:class:`mne.Info` object as source.



In [10]:
sinfo = StreamInfo(
    name="Prediction_stream",
    stype="Labels",
    n_channels=1,
    sfreq=0,
    dtype="string",
    source_id=uuid.uuid4().hex[:6],
)


Once the :class:`~mne_lsl.lsl.StreamInfo` object is created, a
:class:`~mne_lsl.lsl.StreamOutlet` can be instantiated to create the stream.



In [11]:
sinfo

< sInfo 'Prediction_stream' >
  | Type: Labels
  | Sampling: Irregular
  | Number of channels: 1
  | Data type: string
  | Source: 41af08

In [12]:
outlet = StreamOutlet(sinfo)

## Push/Pull operations

For new data to be received, it first need to be pushed on the
:class:`~mne_lsl.lsl.StreamOutlet`. 2 methods are available:

* :meth:`~mne_lsl.lsl.StreamOutlet.push_sample` to push an individual sample of shape
  (n_channels,)
* :meth:`~mne_lsl.lsl.StreamOutlet.push_chunk` to push a chunk of samples of shape
  (n_samples, n_channels)



In [13]:
(np.array([1, 2, 3]))

array([1, 2, 3])

Once pushed, samples become available at the client end. 2 methods are available to
retrieve samples:

* :meth:`~mne_lsl.lsl.StreamInlet.pull_sample` to pull an individual sample of shape
  (n_channels,)
* :meth:`~mne_lsl.lsl.StreamInlet.pull_chunk` to pull a chunk of samples of shape
  (n_samples, n_channels)



In [27]:
Info = mne.create_info (sinfo.n_channels,sinfo.sfreq,'eeg')

# Ensure the channel names are in the expected format (0 to 66)
original_channel_names = [f'{i}' for i in range(67)]  # Adjust if necessary

# New channel names you provided
new_channel_names = [
    "Fp1", "Fz", "F3", "F7", "FT9", "FC5", "FC1", "C3", "T7",
    "CP5", "CP1", "Pz", "P3", "P7", "TP9", "O1", "Oz", "O2", "TP10",
    "P8", "P4", "CP2", "CP6", "T8", "C4", "Cz", "FC2", "FC6", "FT10",
    "F8", "F4", "Fp2", "AF7", "AF3", "AFz", "F1", "F5", "FT7", "FC3",
    "TP7", "C5", "C1", "CP3", "P1", "P5", "PO7", "PO3", "POz", "PO4",
    "PO8", "P6", "P2", "CPz", "CP4", "TP8", "C6", "C2", "FC4", "FT8",
    "F6", "AF8", "AF4", "F2", "Iz", "ACC_X", "ACC_Y", "ACC_Z"
]

# Check if the original file has extra channels or is missing any, and adjust accordingly
#assert len(Raw.ch_names) >= len(original_channel_names), "The raw file has fewer channels than expected."

# Mapping from original to new names
channel_mapping = dict(zip(original_channel_names, new_channel_names))
montage = mne.channels.read_custom_montage((f"{current_path}\Montages\CACS-64_REF.bvef"), head_size=0.095, coord_frame=None) 


In [28]:
import pickle 
fname = 'Noam_05_05'+'_FBCSP_model'

path_fname = current_path /'Models'/ fname
#%% Load the selected model

#read the pickle file   
picklefile = open(path_fname, 'rb')
#unpickle the dataframe
trained_clf = pickle.load(picklefile)
#close file
picklefile.close()
fname = 'electrode_picks-274'
path_fname = current_path /'Models'/ fname
#create a pickle file
picklefile = open(path_fname, 'rb')
#pickle the dictionary and write it to file
picks = pickle.load(picklefile)
#close the file

picklefile.close()

fname = 'mean'
path_fname = current_path /'Models'/ fname
#create a pickle file
picklefile = open(path_fname, 'rb')
#pickle the dictionary and write it to file
mean = pickle.load(picklefile)
#close the file

picklefile.close()

<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.


To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


c:\Users\CensorLab\anaconda3\envs\BCIEnvironment\lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(
c:\Users\CensorLab\anaconda3\envs\BCIEnvironment\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearDiscriminantAnalysis from version 1.2.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\CensorLab\anaconda3\envs\BCIEnvironment\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.2.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/mo

Stuff for FBCSP

In [29]:
#extract filterbank feequencies:

filters_bands=tuple([[8, 12], [12, 20], [20, 28], [28, 32]])
filtered_data_band_passed=[]
for i,(LowPass,HighPass) in enumerate(filters_bands):
    unfiltered_Raw_CSD=Raw_CSD.copy()
    Raw_CSD_Filtered_band= unfiltered_Raw_CSD.filter(LowPass, HighPass, method='fir')
    filtered_data_band_passed.append(Raw_CSD_Filtered_band)

NameError: name 'Raw_CSD' is not defined

In [ ]:
from concurrent.futures import ThreadPoolExecutor
# Function to apply band-pass filter
def filter_band(raw_csd, band):
    LowPass, HighPass = band
    return raw_csd().filter(LowPass, HighPass, method='fir', copy = True)

In [30]:
# Function to add an annotation to the stream_annotations
def add_stream_annotation(onset, description, duration=0):
    new_annotation = mne.Annotations(onset=[onset], duration=[duration], description=[description])
    stream_annotations += new_annotation  # This merges the new annotation with the existing ones


In [31]:
# give a bit of time to the documentation build after the execution of the last cell
inlet.flush()
time.sleep(0.2)
concat_data = np.empty((0, 67))  # Initialize with zero samples but correct channel count
storage_concat_data = np.empty((0, 67))  # Initialize with zero samples but correct channel count
#assert inlet.samples_available == 1
# Initialize an empty Annotations object
stream_annotations = mne.Annotations(onset=[], duration=[], description=[])

duration = 100
predictions =[]
predictions_proba = []
max_samples = 55
start_time = time.time()
while time.time() - start_time < duration:
    while(inlet.samples_available<56):
        time.sleep(0.001)
    data,ts = inlet.pull_chunk(max_samples=max_samples)
    if ((concat_data.shape)[0]>6000):
        concat_data = concat_data[-2500:]  # Reset 'concat_data' after a while for preprocessing 
    concat_data = np.concatenate((concat_data, data), axis=0)
    now = local_clock()
    print(f"Timestamp of the acquired data: {ts[max_samples-1]}")
    print(f"Current time: {now}")
    print(f"Delta: {now - ts[max_samples-1]} seconds")
    
    # Temporary suppress logging output
    original_log_level = logging.getLogger('mne').getEffectiveLevel()
    logging.getLogger('mne').setLevel(logging.ERROR)  # Suppress messages below ERROR level

    Raw=mne.io.RawArray(concat_data.T,Info)

    # Calculate the time of the last sample
    last_sample_time = (Raw.n_times - 1) / Raw.info['sfreq']

    # Rename channels
    Raw.rename_channels(channel_mapping)
    #mne.rename_channels(Raw.info, {'F9' : 'FT9','P9' : 'TP9','P10' : 'TP10','F10' : 'FT10','AF1' : 'AF7' }, allow_duplicates=False, verbose=None)
    Raw.drop_channels(['ACC_X','ACC_Y','ACC_Z','TP7']) ## Drop non eeg channels
    Raw.set_montage(montage, match_case=True, match_alias=False, on_missing='raise', verbose=None)
    Raw._data /= 1e6
    Raw_CSD = mne.preprocessing.compute_current_source_density(Raw) ## Compute CSD
    Raw_CSD.pick(picks)
    unfiltered_Raw = Raw_CSD.copy()
    #Raw_CSD_Filtered = Raw_CSD.filter(5, 34, method='fir')
    
    filtered_data_band_passed=[]
    for i,(LowPass,HighPass) in enumerate(filters_bands):
        Raw_CSD_Filtered_band= mne.filter.filter_data(unfiltered_Raw.get_data(),sfreq = 500, l_freq = LowPass, h_freq=HighPass, method='fir',copy = True)
        filtered_data_band_passed.append(Raw_CSD_Filtered_band)
    
    #Output_data = Raw_CSD_Filtered.get_data()

    Output_data= np.transpose(np.array(filtered_data_band_passed),(1,2,0))
    Output_data = Output_data[:, -1002:,:]
    Output_data = Output_data[np.newaxis, :, :,:]

    # Determine the total number of samples
    #n_samples = Raw_CSD_Filtered.n_times

    # We want the last X samples
    #samples_to_get = 251

    # Calculate the start sample to get the last 55 samples
    #start_sample = n_samples - samples_to_get
    # Use the .get_data() method to get the last 55 samples
    # Assuming you want to get data for all channels, otherwise specify picks=[channel_indices]
    #data_for_classification = filtered_data_band_passed.get_data(start=start_sample, stop=n_samples)
    data_for_classification = Output_data

    # Restore original logging level
    logging.getLogger('mne').setLevel(original_log_level)
    prediction = trained_clf.predict(data_for_classification)
    prediction_proba = trained_clf.predict_proba(data_for_classification)
    #print (prediction)

    if len(predictions)>1:
        if (predictions[-1:][0][0] != prediction[0]):
            # Create an annotation at the last sample time
            new_annotations = mne.Annotations(onset=[last_sample_time], duration=[0], description=[prediction[0]])
            # Add the annotation to the Raw object
            stream_annotations += new_annotations  # This merges the new annotation with the existing ones
            print('got here')

    predictions.append(prediction)
    predictions_proba.append(prediction_proba)
    # Push processed data chunk for classification
    outlet.push_sample(list(prediction[0][-2]))
    #assert inlet.samples_available == 0


Timestamp of the acquired data: 122511.9279844
Current time: 122512.0216945
Delta: 0.09371010000177193 seconds
Timestamp of the acquired data: 122512.0379844
Current time: 122512.1410189
Delta: 0.10303450000355951 seconds
Timestamp of the acquired data: 122512.1479844
Current time: 122512.1934792
Delta: 0.045494799996959046 seconds
Timestamp of the acquired data: 122512.2579844
Current time: 122512.2621405
Delta: 0.004156099996180274 seconds
Timestamp of the acquired data: 122512.3679844
Current time: 122512.3719076
Delta: 0.003923199998098426 seconds
Timestamp of the acquired data: 122512.4779846
Current time: 122512.4814328
Delta: 0.0034482000046409667 seconds
got here
Timestamp of the acquired data: 122512.5879844
Current time: 122512.5920261
Delta: 0.004041699998197146 seconds
Timestamp of the acquired data: 122512.6979847
Current time: 122512.7026586
Delta: 0.004673899995395914 seconds
Timestamp of the acquired data: 122512.8079845
Current time: 122512.8112981
Delta: 0.00331359999

KeyboardInterrupt: 

In [32]:
predictions

[array(['Resting'], dtype='<U7'),
 array(['Left'], dtype='<U7'),
 array(['Left'], dtype='<U7'),
 array(['Left'], dtype='<U7'),
 array(['Left'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Resting'], dtype='<U7'),
 array(['Left'], dtype='<U7'),
 array(['Left'], dtype='<U7'),
 array(['Right'], dtype='<U7'),
 array(['Left'], dtype='<U7'),
 array(['Left'], dtype='<U7'),
 array(['Left'], dtype='<U7'),
 array(['Left'], dtype='<U7'),
 array(['Left'

In [69]:

    Raw.set_annotations(stream_annotations)
    Raw_for_analysis.plot()


In [63]:
    Raw_for_analysis.set_annotations(stream_annotations)
    Raw_for_analysis.plot()

In [29]:
predictions_proba

[array([[4.35163072e-14, 1.00000000e+00, 7.84815158e-13]]),
 array([[2.29819141e-06, 9.99988955e-01, 8.74688123e-06]]),
 array([[0.80726741, 0.13619253, 0.05654006]]),
 array([[0.97317917, 0.01134348, 0.01547735]]),
 array([[0.94513928, 0.01610752, 0.0387532 ]]),
 array([[0.74363352, 0.0735587 , 0.18280779]]),
 array([[0.89733893, 0.01794598, 0.08471508]]),
 array([[0.62539582, 0.0439906 , 0.33061358]]),
 array([[0.66279908, 0.12053803, 0.21666289]]),
 array([[0.63960509, 0.10826489, 0.25213002]]),
 array([[0.45670566, 0.3621206 , 0.18117373]]),
 array([[0.07976596, 0.76774966, 0.15248438]]),
 array([[0.06559015, 0.81627191, 0.11813794]]),
 array([[0.07649235, 0.75728029, 0.16622736]]),
 array([[0.14782541, 0.63988166, 0.21229293]]),
 array([[0.17889464, 0.66584075, 0.1552646 ]]),
 array([[0.18073695, 0.56532   , 0.25394305]]),
 array([[0.23937772, 0.45140028, 0.309222  ]]),
 array([[0.28404613, 0.39395465, 0.32199921]]),
 array([[0.32396532, 0.39857258, 0.2774621 ]]),
 array([[0.32296

In [185]:
Raw.plot()

In [11]:
Info = mne.create_info (sinfo.n_channels,sinfo.sfreq,'eeg')
Raw=mne.io.RawArray(concat_data.T,Info)

NameError: name 'concat_data' is not defined

In [152]:
filtered_data_band_passed

[<RawArray | 20 x 10120 (20.2 s), ~1.6 MB, data loaded>,
 <RawArray | 20 x 10120 (20.2 s), ~1.6 MB, data loaded>,
 <RawArray | 20 x 10120 (20.2 s), ~1.6 MB, data loaded>,
 <RawArray | 20 x 10120 (20.2 s), ~1.6 MB, data loaded>]

In [19]:
Raw._data /= 1e6

In [184]:

Raw_CSD_Filtered.plot()

In [153]:
filtered_data_band_passed = []
for i,(LowPass,HighPass) in enumerate(filters_bands):
    unfiltered_Raw_CSD=Raw_CSD_Filtered.copy()
    Raw_CSD_Filtered_band= unfiltered_Raw_CSD.filter(LowPass, HighPass, method='fir')
    filtered_data_band_passed.append(Raw_CSD_Filtered_band)



AttributeError: 'list' object has no attribute 'get_data'

In [12]:
# Define paths: 
current_path = pathlib.Path().absolute()  
recording_path = current_path / 'Recordings'

OriginalRaw=read_raw_xdf(recording_path / 'NH_Block_3.xdf')


NameError: name 'read_raw_xdf' is not defined

In [64]:
unfiltered_Raw=Raw.copy()
unfiltered_OriginalRaw=OriginalRaw.copy()

Raw_Filtered = unfiltered_Raw.filter(8, 32, method='fir')
OriginalRaw_Filtered = unfiltered_OriginalRaw.filter(8, 32, method='fir')


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 32 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 32.00 Hz
- Upper transition bandwidth: 8.00 Hz (-6 dB cutoff frequency: 36.00 Hz)
- Filter length: 825 samples (1.650 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 32 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 32.00 Hz
- Upper transition bandwidth: 8.00 Hz (-6 dB cutoff frequency: 36.00 Hz)
- Filter length: 825 samples (1.650 s)



[Parallel(n_jobs=1)]: Done  67 out of  67 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  67 out of  67 | elapsed:    0.6s finished


In [61]:

# Assuming 'raw' is your Raw MNE data instance
sfreq = 500  # Sampling frequency
tmin = 80  # Start time in seconds
tmax = 105  # End time in seconds

# Calculate start and end sample indices
start_sample = int(tmin * sfreq)
end_sample = int(tmax * sfreq)

# Crop the raw data
OriginalRawCropped = OriginalRaw_Filtered.copy().crop(tmin=tmin, tmax=tmax)

# Now 'raw_cropped' contains the segment of the data from 90 to 115 seconds


In [44]:
OriginalRawCroppedData = OriginalRawCropped.get_data()
LSLData = Raw.get_data()

In [60]:
unfiltered_Raw=Raw.copy()
unfiltered_OriginalRaw=OriginalRaw.copy()

Raw_Filtered = unfiltered_Raw.filter(8, 32, method='iir')
OriginalRaw_Filtered = unfiltered_OriginalRaw.filter(8, 32, method='iir')


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 32 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 8.00, 32.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 32 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 8.00, 32.00 Hz: -6.02, -6.02 dB



In [64]:
OriginalRawCroppedData = OriginalRawCropped.get_data()
LSLData = Raw_Filtered.get_data()

In [65]:
display(OriginalRawCroppedData[0,:])

array([-2.05663489e-06, -1.62567150e-06, -9.81545734e-07, -1.77730737e-07,
        7.12710861e-07,  1.60916388e-06,  2.43667556e-06,  3.13736926e-06,
        3.67749743e-06,  4.04879166e-06,  4.26438991e-06,  4.35106154e-06,
        4.34020119e-06,  4.25992787e-06,  4.12980060e-06,  3.95861057e-06,
        3.74487173e-06,  3.47918372e-06,  3.14750699e-06,  2.73445192e-06,
        2.22592100e-06,  1.61086603e-06,  8.82433267e-07,  3.91483631e-08,
       -9.13186995e-07, -1.95862862e-06, -3.06769785e-06, -4.19460244e-06,
       -5.27659343e-06, -6.23686196e-06, -6.99171276e-06, -7.46165118e-06,
       -7.58475841e-06, -7.32967177e-06, -6.70500305e-06, -5.76238053e-06,
       -4.59153847e-06, -3.30773987e-06, -2.03376073e-06, -8.80091464e-07,
        7.24787938e-08,  7.84241799e-07,  1.25964471e-06,  1.54170121e-06,
        1.69933113e-06,  1.81068797e-06,  1.94613748e-06,  2.15421535e-06,
        2.45286919e-06,  2.82699306e-06,  3.23204834e-06,  3.60261009e-06,
        3.86402425e-06,  

In [59]:
display(LSLData[0,:])

array([-4.23633359e-09,  7.97463193e-07,  1.37292660e-06,  1.54129221e-06,
        1.20371265e-06,  3.64042635e-07, -8.70299372e-07, -2.30731870e-06,
       -3.70545740e-06, -4.81778963e-06, -5.43695147e-06, -5.43244810e-06,
       -4.77369435e-06, -3.53506973e-06, -1.88274619e-06, -4.63174873e-08,
        1.71931957e-06,  3.17512209e-06,  4.13526545e-06,  4.49474970e-06,
        4.24300891e-06,  3.46169500e-06,  2.30754142e-06,  9.83763778e-07,
       -2.94660519e-07, -1.33562965e-06, -1.99859888e-06, -2.21289019e-06,
       -1.98343518e-06, -1.38415575e-06, -5.41191898e-07,  3.90516149e-07,
        1.25321993e-06,  1.91090585e-06,  2.26889333e-06,  2.28589564e-06,
        1.97715165e-06,  1.40862980e-06,  6.83699921e-07, -7.52011795e-08,
       -7.46752831e-07, -1.22955957e-06, -1.45675668e-06, -1.40430322e-06,
       -1.09188748e-06, -5.76717318e-07,  5.80965045e-08,  7.17812860e-07,
        1.30999908e-06,  1.75574622e-06,  1.99687327e-06,  1.99900122e-06,
        1.75128733e-06,  

In [ ]:
Raw_Filtered.plot()

Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none


: 

In [38]:

def array_in_array(small, large):
    for i in range(large.shape[0] - small.shape[0] + 1):
        for j in range(large.shape[1] - small.shape[1] + 1):
            # Extracting the slice of the larger array to compare
            slice_of_large = large[i:i+small.shape[0], j:j+small.shape[1]]
            if np.array_equal(slice_of_large, small):
                return True, (i, j)  # Found the small array, return True and the starting position
    return False, None  # Small array not found

# This is just a demonstration and might not be practical for very large arrays
exists, position = array_in_array(DS [ 2:5, :], Raw.get_data())
print(f"Exists: {exists}, Position: {position}")

Exists: True, Position: (2, 0)


## LSL clock

The local system timestamp is retrieved with :func:`~mne_lsl.lsl.local_clock`. This
local timestamp can be compared with the LSL timestamp from acquired data.



In [48]:
now = local_clock()
print(f"Timestamp of the acquired data: {ts}")
print(f"Current time: {now}")
print(f"Delta: {now - ts} seconds")

Timestamp of the acquired data: 62361.6523904
Current time: 62382.5415168
Delta: 20.889126400004898 seconds


In [55]:
inlet.samples_available

5482

## Free resources

When you are done with a :class:`~mne_lsl.lsl.StreamInlet` or
:class:`~mne_lsl.lsl.StreamOutlet`, don't forget to free the resources they both use.



In [37]:
inlet.close_stream()
del inlet


NameError: name 'inlet' is not defined